In [ ]:
# kütüphaneler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
import seaborn as sns            
import os                        

import torch
import torch.nn as nn            
import torch.optim as optim      

import torchvision
from torchvision import datasets, models, transforms 

from sklearn.metrics import classification_report, confusion_matrix

from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan Cihaz: {device}")


In [ ]:
# data preprocessing
import torch
import os
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset, Dataset, ConcatDataset

# Resnet renk standardı
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]
# veriyi modele vermek üzere 2 ye böldük train yani eğitileceği data ve val yani test edileceği data
# train kısmındaki fotoğraflara çevirme, döndürme, ışık ayarlarını değiştirme gibi değişiklikler uygulayarak modelin
# ezberlemesini zorlaştırıyoruz
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(30),
        transforms.ColorJitter(brightness= 0.2, contrast= 0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean_nums, std_nums)
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean_nums, std_nums)
    ]),
}

base_dir = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)'
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')

hedef_bitkiler = ["Tomato", "Apple", "Grape", "Corn"]

# datasetteki tüm verileri kullanmadığımız için veriyi alırken etikette sıkıntı çıkıyordu onun için yardımcı 
class RemappedSubset(Dataset):
    def __init__(self, subset, old_to_new_mapping):
        self.subset = subset
        self.mapping = old_to_new_mapping
    
    def __len__(self):
        return len(self.subset)
    
    def __getitem__(self, idx):
        image, old_label = self.subset[idx]
        new_label = self.mapping[old_label]
        return image, new_label

class NaturalImagesDataset(Dataset):
    def __init__(self, root_dir, transform, label_idx, limit=600):

        full_dataset = datasets.ImageFolder(root=root_dir, transform=transform)
        
        total_images = len(full_dataset)

        actual_limit = min(limit, total_images)

        indices = torch.randperm(total_images)[:actual_limit]

        self.dataset = Subset(full_dataset, indices)
        self.label_idx = label_idx
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        # Subset içinden resmi al
        image, _ = self.dataset[idx] 
        # Orijinal etiketi (_) çöpe at, bizim 'Others' etiketini ver
        return image, self.label_idx

def filtrele_ve_olustur(ana_klasor_yolu, transform_tipi):
    full_dataset = datasets.ImageFolder(
        root=ana_klasor_yolu,
        transform=data_transforms[transform_tipi]
    )
    
    # Hedef sınıfların eski indekslerini bul
    hedef_sinif_indeksleri = []
    for i, sinif_adi in enumerate(full_dataset.classes):
        for bitki in hedef_bitkiler:
            if bitki in sinif_adi:
                hedef_sinif_indeksleri.append(i)
                break 
    
    # Bu sınıflara ait resimlerin indekslerini topla
    secilen_resim_indeksleri = [i for i, label in enumerate(full_dataset.targets) 
                                if label in hedef_sinif_indeksleri]
    
    # Subset oluştur
    subset_dataset = Subset(full_dataset, secilen_resim_indeksleri)
    
    # Etiketleri düzenliyoruz(0,1,2...)
    hedef_sinif_indeksleri_sorted = sorted(hedef_sinif_indeksleri)
    old_to_new = {old_idx: new_idx for new_idx, old_idx in enumerate(hedef_sinif_indeksleri_sorted)}
    
    # Remapped dataset oluştur
    remapped_dataset = RemappedSubset(subset_dataset, old_to_new)
    
    # Seçilen sınıf isimlerini de döndür
    secilen_sinif_isimleri = [full_dataset.classes[i] for i in hedef_sinif_indeksleri_sorted]
    
    return remapped_dataset, secilen_sinif_isimleri

# Eğitim seti (Train klasöründen)
train_set_plants, bitki_siniflari = filtrele_ve_olustur(train_dir, 'train')

# Test seti (Valid klasöründen)
val_set_plants, _ = filtrele_ve_olustur(valid_dir, 'val')

others_label_idx = len(bitki_siniflari)

natural_img_path = '/kaggle/input/natural-images/natural_images' 

# Train için 
natural_train = NaturalImagesDataset(
    root_dir=natural_img_path, 
    transform=data_transforms['train'], 
    label_idx=others_label_idx, 
    limit=600
)

# Valid için 
natural_val = NaturalImagesDataset(
    root_dir=natural_img_path, 
    transform=data_transforms['val'], 
    label_idx=others_label_idx, 
    limit=150
)

# Bitki verisi ile Natural Images verisini birleştir
train_set = ConcatDataset([train_set_plants, natural_train])
val_set = ConcatDataset([val_set_plants, natural_val])

# Sınıf listesine Others'ı ekle
bitki_siniflari.append("Others")
# ---------------------------------------------

# DataLoader oluştur
train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False, num_workers=2)

# Bilgi yazdır
print("-" * 50)
print(f"Toplam Eğitim Resmi: {len(train_set)}")
print(f"Toplam Test Resmi:   {len(val_set)}")
print(f"Tespit Edilecek Sınıf Sayısı: {len(bitki_siniflari)}")
print("-" * 50)
print("Seçilen Sınıflar:")
for i, s in enumerate(bitki_siniflari):
    print(f" [{i}] {s}")
print("-" * 50)

# Etiketlerin doğru olduğunu doğrula
images, labels = next(iter(train_loader))
print(f"Etiket Kontrolü:")
print(f"   Min etiket: {labels.min().item()}")
print(f"   Max etiket: {labels.max().item()}")
print(f"   Benzersiz etiketler: {sorted(labels.unique().tolist())}")
print(f"   Beklenen aralık: 0-{len(bitki_siniflari)-1}")
print("-" * 50)

# Model için sınıf sayısı
num_classes = len(bitki_siniflari)

In [ ]:
# Model inşası
import torch.nn as nn
from torchvision.models import ResNet50_Weights

num_classes = len(bitki_siniflari)

# 1. Modeli İndir
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# 2. Son katman değişimi
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# 3. GPU'ya Gönder
model = model.to(device)

In [ ]:
# Eğitim
import torch.optim as optim
import time

images, labels = next(iter(train_loader))
assert labels.min() >= 0, f"Negatif etiket: {labels.min()}"
assert labels.max() < model.fc.out_features, f"Etiket {labels.max()} çok büyük! Model {model.fc.out_features} sınıf bekliyor."
print("Etiketler doğru!")

# Hata Hesaplayıcı: Sınıflandırma olduğu için CrossEntropy kullanıyoruz.
criterion = nn.CrossEntropyLoss()

# Optimizer (Güncelleyici): Modelin her adımda ne kadar güncelleme yapacağı(kendini düzelteceği)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Eğitim fonksiyonu
def train_model(model, criterion, optimizer, num_epochs=10):
    start_time = time.time()
    train_losses = [] # Grafik çizmek için hatayı sakla
    
    print(f"Toplam {num_epochs} tur atılacak.")
    
    for epoch in range(num_epochs):
        model.train() # Modeli antrenman moduna al
        running_loss = 0.0
        correct = 0
        total = 0
        
        # Veriyi çekiyoruz
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Sıfırla
            optimizer.zero_grad()
            
            # Model tahmin etsin
            outputs = model(inputs)
            
            # Güncellemek üzere loss u hesapla
            loss = criterion(outputs, labels)
            
            # Geriye bak
            loss.backward()
            
            # Güncelle (öğrenme kısmı)
            optimizer.step()
            
            # İstatistik
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        # Her epoch sonunda rapor
        epoch_loss = running_loss / len(train_set)
        epoch_acc = 100 * correct / total
        train_losses.append(epoch_loss)
        
        print(f"Tur [{epoch+1}/{num_epochs}] Bitti -> Hata: {epoch_loss:.4f} | Başarı: %{epoch_acc:.2f}")

    end_time = time.time() - start_time
    print(f"Eğitim Tamamlandı! Süre: {end_time // 60:.0f}dk {end_time % 60:.0f}sn")
    return train_losses

# Eğitimi başlat
loss_history = train_model(model, criterion, optimizer, num_epochs=10)

# Kaydet
torch.save(model.state_dict(), 'model_with_natural_images.pth')
print("Model dosyası kaydedildi: model_with_natural_images.pth")

In [ ]:
# Değerlendirme ve Raporlama
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import torch
import glob
import random
from PIL import Image
import os

plt.figure(figsize=(10, 5))
plt.plot(loss_history, label='Eğitim Hatası (Training Loss)')
plt.title('Modelin Öğrenme Grafiği')
plt.xlabel('Epoch Sayısı')
plt.ylabel('Hata Değeri')
plt.legend()
plt.grid(True)
plt.show()

print("Model test ediliyor, lütfen bekleyin...")
tum_tahminler = []
tum_gercekler = []

model.eval() 
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        tum_tahminler.extend(preds.cpu().numpy())
        tum_gercekler.extend(labels.numpy())

try:
    target_names = bitki_siniflari
except:
    target_names = full_dataset.classes

print("\n--- DETAYLI SINIFLANDIRMA RAPORU ---")
print(classification_report(tum_gercekler, tum_tahminler, target_names=target_names))

cm = confusion_matrix(tum_gercekler, tum_tahminler)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=target_names, 
            yticklabels=target_names)
plt.xlabel('Modelin Tahmini')
plt.ylabel('Gerçek Durum')
plt.title('Confusion Matrix (Hata Haritası)')
plt.xticks(rotation=90) 
plt.show()

# SINIF DAĞILIMI GRAFİĞİ (Bar Chart)
print("Veri seti dağılımı hesaplanıyor (Biraz sürebilir)...")

# DataLoader'dan gerçek sayıları al
label_counts = torch.zeros(len(bitki_siniflari))
with torch.no_grad():
    for _, targets in train_loader:
        for target in targets:
            label_counts[target] += 1

counts_np = label_counts.numpy().astype(int)

# Grafiği Çiz
plt.figure(figsize=(14, 6))
ax = sns.barplot(x=bitki_siniflari, y=counts_np, palette="viridis")

# Sayıları çubukların üstüne yaz
for i, v in enumerate(counts_np):
    ax.text(i, v + 5, str(v), color='black', fontweight='bold', ha='center', fontsize=9)

plt.title('Eğitim Veri Seti Sınıf Dağılımı (Bitkiler + Others)', fontsize=15)
plt.ylabel('Fotoğraf Sayısı')
plt.xticks(rotation=90) # İsimler sığsın 
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# ÖRNEK FOTOĞRAFLAR (Bitkiler + Others)
# Göstermek istediğimiz bitkiler
visual_targets = ["Tomato", "Apple", "Grape", "Corn"]

# Tablo boyutunu ayarla
fig, axes = plt.subplots(len(visual_targets) + 1, 2, figsize=(10, 18))
fig.suptitle('Veri Setinden Rastgele Örnekler', fontsize=16)

# A) BİTKİLERİ GÖSTER (Sağlıklı vs Hasta)
for i, plant in enumerate(visual_targets):
    # Klasör desenleri
    healthy_pattern = os.path.join(train_dir, f"{plant}*healthy*")
    disease_pattern = os.path.join(train_dir, f"{plant}*")
    
    # Klasörleri bul
    healthy_folders = glob.glob(healthy_pattern)
    all_folders = glob.glob(disease_pattern)
    disease_folders = [f for f in all_folders if "healthy" not in f]
    
    # 1. Sağlıklı Resim (Sol Sütun)
    if healthy_folders:
        # Klasörün içindeki jpg'leri bul
        img_files = glob.glob(os.path.join(healthy_folders[0], "*"))
        if img_files:
            img = Image.open(random.choice(img_files))
            axes[i, 0].imshow(img)
            axes[i, 0].set_title(f"{plant} - Sağlıklı")
    axes[i, 0].axis('off')

    # 2. Hastalıklı Resim (Sağ Sütun)
    if disease_folders:
        random_disease_folder = random.choice(disease_folders)
        disease_name = os.path.basename(random_disease_folder).split("___")[-1].replace("_", " ")
        
        img_files = glob.glob(os.path.join(random_disease_folder, "*"))
        if img_files:
            img = Image.open(random.choice(img_files))
            axes[i, 1].imshow(img)
            axes[i, 1].set_title(f"{plant} - {disease_name}")
    axes[i, 1].axis('off')

# B) OTHERS (Natural Images) 
row_idx = len(visual_targets)

# Natural Images klasöründen rastgele iki sınıf seç
if os.path.exists(natural_img_path):
    others_categories = os.listdir(natural_img_path) 
    
    # Sol Sütun (Rastgele Others 1)
    cat1 = random.choice(others_categories)
    img_files1 = glob.glob(os.path.join(natural_img_path, cat1, "*"))
    if img_files1:
        img = Image.open(random.choice(img_files1))
        axes[row_idx, 0].imshow(img)
        axes[row_idx, 0].set_title(f"Others (Örn: {cat1})")
    
    # Sağ Sütun (Rastgele Others 2)
    cat2 = random.choice(others_categories)
    img_files2 = glob.glob(os.path.join(natural_img_path, cat2, "*"))
    if img_files2:
        img = Image.open(random.choice(img_files2))
        axes[row_idx, 1].imshow(img)
        axes[row_idx, 1].set_title(f"Others (Örn: {cat2})")

axes[row_idx, 0].axis('off')
axes[row_idx, 1].axis('off')

plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()

# RAPORDA KULLANILMADI (çok fazla veri olduğu için yüzdelik sayıların okunması zor)
# Mevcut 'cm' (Confusion Matrix) değişkenini kullanır.
# Köşegen elemanları (doğru tahminler) / Toplam satır sayısı (gerçek adet)
class_accuracy = cm.diagonal() / cm.sum(axis=1)

plt.figure(figsize=(12, 6))
# Renk paleti
colors = sns.color_palette("husl", len(bitki_siniflari))
bars = plt.bar(bitki_siniflari, class_accuracy, color=colors)

plt.title('Sınıf Bazlı Doğruluk Oranları (Per-Class Accuracy)', fontsize=15)
plt.ylabel('Başarı Oranı (0.0 - 1.0)', fontsize=12)
plt.xlabel('Sınıflar', fontsize=12)
plt.xticks(rotation=45)
plt.ylim(0, 1.1) 

# Çubukların üzerine yüzdeleri yaz
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.02,
             f'%{height*100:.1f}',
             ha='center', va='bottom', fontweight='bold')

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Matrisi yüzdelik hale getir (Her satırı kendi toplamına böl)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(12, 10))
sns.heatmap(cm_normalized, annot=True, fmt='.1%', cmap='Blues', vmin=0, vmax=1,
            xticklabels=bitki_siniflari, 
            yticklabels=bitki_siniflari)
plt.xlabel('Modelin Tahmini', fontsize=12)
plt.ylabel('Gerçek Durum', fontsize=12)
plt.title('Normalize Edilmiş Confusion Matrix (Yüzdelik Başarı)', fontsize=15)
plt.xticks(rotation=90) 
plt.tight_layout()
plt.show()